#  U-Net Test: *Single-class* CT Bone Segmentation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random

import torch
from torch.utils.data import DataLoader, random_split
from torch.optim import lr_scheduler

from model import UNet
from dataset import CTMaskDataset
from losses import FocalLoss, MixedLoss, dice, IoU
from train import train_net
from eval import eval_net
from predict import predict
from utils import (readFloatImage, readBinImage, readUCharImage, 
                   plotSomeImages, matchFilesFromPatient)

from datetime import datetime

## Define Datasets
Get the CT and label mask files into Datasets.

In [ ]:
# get patient data list from nested file system
patient_idxs = [1, 3, 4, 5, 9]
ct_data = []
for idx in patient_idxs:
    for day_selection in range(1,4):
        matched_data = matchFilesFromPatient(idx, 
                                             day_selection, 
                                             mode='CT_SPINE',
                                             no_empties=True)
        ct_data.extend(matched_data)

random.shuffle(ct_data)

# set training and validation size
train_set_size = 1000
val_set_size = int(0.2 * train_set_size)
train_data = ct_data[0:train_set_size]
val_data = ct_data[train_set_size:train_set_size + val_set_size]

train_dataset = CTMaskDataset(train_data, augment=True)
val_dataset = CTMaskDataset(val_data, augment=False)

batch_size = 5
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=8)

val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=True,
                        num_workers=8)

## Dataset verification
Check the dataset containers for correct data and shape.

In [ ]:
idx = np.random.choice(len(train_dataset))
sample_data = train_dataset[idx]

ct = sample_data['image'][0,:,:]
mask = sample_data['target'][0,:,:]

print('data sample:')
print('ct image shape: ' + str(ct.shape))
print(sample_data['image'])
print('mask image shape: ' + str(mask.shape))
print(sample_data['target'])

images = {'image': ct,
          'target': mask
         }
plotSomeImages(images, 1, 2)

## Create datalog folder

In [ ]:
current_datetime = datetime.now().strftime('%Y.%m.%d-%H.%M.%S')
output_directory = 'trainlog-' + current_datetime
if not os.path.isdir(output_directory):
    os.makedirs(output_directory)

## Training initialization
Define hyperparameters for learning, loss function, optimizer, etc...

In [ ]:
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

# model hyperparameters
lr_model = 0.0003
decay_step_size = 500
focal_gain = 10.0

# declare model
model = UNet(n_channels = 1, 
             n_classes = 1,
             large_model = True)

model.to(device)

# learning schema
criterion = MixedLoss(focal_gain, gamma=2)

optimizer = torch.optim.Adam(model.parameters(), lr=lr_model)

learning_scheduler = lr_scheduler.StepLR(optimizer, 
                                         step_size=decay_step_size, 
                                         gamma=0.1)

num_epochs = 20
ckpt_save_interval = 2

# loss log initialization
train_losses = []
val_losses = []
best_score = 0.0


## Training loop
General training scheme with validation statistics recorded per epoch. Best models are saved.

In [ ]:
for epoch in range(num_epochs):
    # run training step
    train_loss = train_net(model, 
                           device, 
                           train_loader,
                           batch_size,
                           criterion, 
                           optimizer,
                           learning_scheduler,
                           epoch,
                           print_log=True)

    # run validation step
    val_loss = eval_net(model, 
                        device, 
                        val_loader, 
                        print_log=True)

    # save best models
    if val_loss[0] > best_score:
        best_score = val_loss[0]
        torch.save(model.state_dict(), 
                   '{}/unet_model_{}.pt'.format(output_directory, epoch))

    # record losses to logs
    train_losses.append(train_loss)
    val_losses.append(val_loss)

## Training charts
Not sure if this works right now.

In [ ]:
dice_scores, iou_scores = zip(*val_losses)
n = range(0, len(dice_scores))
fig = plt.figure(figsize=(12,8))
plt.plot(n, dice_scores, label='dice')
plt.plot(n, iou_scores, label='iou')
plt.grid('on')
plt.legend()

## Inference test
Grab a random CT image from the loaded dataset and performs a prediction!

In [ ]:
rand_idx = random.randint(1, len(ct_data))
ct_file = ct_data[rand_idx][0]
img = readBinImage(ct_file)

mask_file = ct_data[rand_idx][1]
true_mask = readUCharImage(mask_file)

prediction = predict(model, ct_file)
prediction = prediction[0,0,:,:]
prediction = torch.sigmoid(prediction).detach().cpu().numpy()

images = {'ct': img,
          'truth': true_mask,
          'prediction': prediction
         }
plotSomeImages(images, 1, 3)